# Nächster Nachbar Klassifikator
Man soll ein Buch nicht nach dem Einband bewerten. In welchem Regal es steht kann dagegen mehr verraten, als dem Besitzer lieb ist. Der Nächste Nachbar Klassifikator macht sich diese Eigenschaft zu Nutze, indem er aus Eigenschaften der Umgebung auf die Eigenschaften eines noch nicht klassifizierten Punktes ableitet.

## Inhaltsverzeichnis
- [Beispieldaten](#Beispieldaten)
- [Definition und Beispiel](#Definition-und-Beispiel)
- [k-Nächster-Nachbar-Klassifikator](#k-Nächster-Nachbar-Klassifikator)
- [Approximativer Nächster Nachbar Algorithmus](#Approximativer-Nächster-Nachbar-Algorithmus)
- [Implementierung in `scikit-learn`](#Implementierung-in-scikit-learn)
- [Datennormalisierung](#Datennormalisierung)

## Beispieldaten
Laden Sie zunächst die benötigten Bibliotheken und den künstlichen Beispieldatensatz.

In [ ]:
import pandas as pd
import plotly.express as px

df = pd.read_csv('nearestneighbour.csv')
px.scatter(df, x='x', y='y', color='c')

## Definition und Beispiel
Der Nächste Nachbar Klassifikator ist ein Klassifikationsverfahren, bei dem die Klassenzuordnung anhand der Entfernung zu Punkten mit bekannten Klassen vorgenommen wird. Demnach ist also erneut ein Distanzmaß wie beispielsweise der euklidische Abstand eine Voraussetzung. Bei der Klassifikation eines neuen Punktes wird der Abstand zu jedem bereits bekannten Punkt berechnet. Die Klasse des Punktes, der den geringsten Abstand besitzt, wird auch für den neuen Punkt angenommen.

Im vorangegangenen Abschnitt hatten Sie bereits die Aufgabe, aus Größe und Gewicht einen Schluss zu ziehen. Im Folgenden haben Sie die Möglichkeit, den größten der Punkte in der Ebene zu bewegen und die Klassifikation auf Basis des nächsten Nachbarn in Echtzeit zu erkunden.

In [ ]:
from animation import draw
draw(df)

## k-Nächster-Nachbar-Klassifikator
An einigen Positionen ragen Punkte einer anderen Klasse in eine Häufung hinein. Zu beobachten ist dies beispielsweise links der Bildmitte, wo sich zwei rote Punkte umschlossen von blauen befinden. An dieser Stelle können Sie eine Zuordnung zur roten Klasse erzeugen, obwohl eine Zuordnung zu den blauen Punkten vermutlich sinnvoller wäre. Die beiden roten Punkte stellen dabei Rauschen dar. Durch eine Störung in der Messung oder Ausreißern bei den gemessenen Werten entfernen sich diese weit genug von ihrem eigenen Häufungspunkt, um zu einer anderen Klasse zugehörig zu erscheinen und den Nächsten Nachbar Klassifikator negativ in seiner Genauigkeit zu beeinflussen.

In der Regel sind es jedoch nur vereinzelte Punkte, die so extrem abweichen. Der k-Nächster-Nachbar-Algorithmus macht sich diese Eigenschaft zu Nutze. Wird nun ein neuer Punkt klassifiziert, werden die $k$ nächsten Nachbarn bestimmt und beispielsweise durch Mehrheitsentscheid aus diesen eine Klasse abgeleitet.

In der folgenden Zelle haben Sie erneut die Möglichkeit, den Algorithmus erkunden. Probieren Sie auch einmal $k=1$ aus.

In [ ]:
from ipywidgets import interact, IntSlider

@interact(k=IntSlider(5, 1, 30, 1))
def _(k):
    draw(df, k)

## Approximativer Nächster Nachbar Algorithmus
Weiterhin werden zur Bewertung jedes neu hinzugekommenen Punktes die Abstände zu jedem einzelnen bekannten Punkt berechnet. Zudem müssen immer alle Punkte im Speicher vorgehalten werden. Mit großen Datensätzen, die einige Millionen Punkte in einen mehrdimensionalen Raum legen, kann dies schnell zu einer ressourcenaufwendigen Angelegenheit werden. Es gibt daher verschiedenste Verfahren, um entweder die Menge an Punkten zu reduzieren oder die Suche der nächsten Nachbarn innerhalb dieser zu beschleunigen. An dieser Stelle soll eine der einfachsten Methoden kurz beleuchtet werden.

Mit Hilfe des Ihnen bereits bekannten k-Means-Algorithmus werden eine Menge an Clusterzentren berechnet. Die entstandenen Zentroide dienen anschließend als Menge, aus denen der nächste Nachbar gewählt wird. (Deshalb können auch mehr Cluster berechnet werden, als Klassen vorhanden sind.) Nach einmaliger Berechnung der Clusterzentren ist die Bestimmung der Distanz also folglich auf eine relativ kleine Menge an Punkten reduziert.

In der folgenden Zelle können Sie mit der Anzahl der Clusterzentren experimentieren. Dabei können Sie sogar feststellen, dass der Einfluss von Rauschen durch eine gut gewählte Clusteranzahl reduziert wird.

In [ ]:
from ipywidgets import interact, IntSlider

@interact(c=IntSlider(3, 1, 50, 1))
def _(c):
    draw(df, cluster=c)

## Implementierung in `scikit-learn`
Natürlich müssen Sie auch den Nächsten-Nachbar-Algorithmus nicht selbst implementieren. Eine Funktion in `scikit-learn` steht mit vielen Optimierungen frei zur Verfügung. Der Konstruktor erhält zunächst die Anzahl der Nachbarn, die betrachtet werden sollen. Anschließend müssen Sie der Funktion `fit` eine Menge von Koordinaten und eine Liste der Klassen übergeben. Ein anschließender Aufruf der Funktion `predict` bestimmt die Klasse anhand der angegebenen Anzahl an nächsten Nachbarn.

In [ ]:
# Import
from sklearn.neighbors import KNeighborsClassifier

# fit
nn = KNeighborsClassifier(n_neighbors=3).fit(df[['x', 'y']], df['c'])

# kneighbors
test_data = pd.DataFrame({
    'x': [24, 55],
    'y': [39, 12],
})

nn.predict(test_data)

## Datennormalisierung
Wie bereits beim Clustering ist eine Distanzfunktion die Voraussetzung für den Nächsten-Nachbar-Klassifikator. Erneut ist diese anfällig gegenüber ungleich skalierten Achsen. Sie sollten im Zweifelsfall auch bei der Klassifikation zunächst mit Hilfe eines `MinMaxScaler` oder `StandardScaler` die Achsen normalisieren.

**Ein abschließender Hinweis:** Während der Darstellung kann die Skalierung von Achsen verändert werden, um die reservierte Fläche des Bildschirms besser auszunutzen. Bei einer Berechnung des Abstands auf Basis der ursprünglichen Koordinaten kann es deshalb dazu kommen, dass in der grafischen Darstellung ein Punkt, der nicht als nächster Nachbar erkannt wird, vermeintlich näher liegt. In den interaktiven grafischen Darstellungen wird daher die Pixeldistanz innerhalb der Darstellung verwendet.